In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import re
import nltk   
from urllib.request import urlopen
from nltk.stem import SnowballStemmer

In [3]:
table = pd.read_csv('/content/drive/MyDrive/dataset_clean_balanced_classes.csv', index_col='Unnamed: 0')

In [4]:
table.head()

,TNVED,OPISANIE
3646289,8431,част бульдозер экскаватор грейдер стойк рыхлит...
661713,1704,конфет мармелад ассорт фруктов вкус соста сиро...
3530959,8516,электрооборудован обогрев пространств встроен ...
1844688,3004,лекарствен средств белосалик лосьон раствор на...
3371156,8516,тепловентилятор бытов напольн электрическ отоп...


In [5]:
!pip install stop-words

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
from stop_words import get_stop_words
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

#### Теперь векторизуем методом TfidfVectorizer

In [7]:
import json
text = table['OPISANIE']
vectorizer = TfidfVectorizer(max_df=0.95, min_df=5, encoding='utf-8')
X_TfidfVectorizer = vectorizer.fit_transform(table['OPISANIE'].values.astype('U'))
# with open('voc.json', 'w') as f:
  # json.dump(vectorizer.get_feature_names(), f)

In [8]:
vocabulary_TfidfVectorizer = vectorizer.get_feature_names()
print('Размер словаря:', len(vocabulary_TfidfVectorizer))
print(vocabulary_TfidfVectorizer)
with open('voc.json', 'w') as f:
  json.dump(vocabulary_TfidfVectorizer, f)

Размер словаря: 18938
['аа', 'абак', 'абатмент', 'абердин', 'абзац', 'абиратерон', 'абонент', 'абонентск', 'абр', 'абраз', 'абразив', 'абразивн', 'абрикос', 'абрикосов', 'абс', 'абсолютирова', 'абсолютн', 'абсорбент', 'абсорбер', 'абсорбир', 'абсорбирова', 'абсорбц', 'абсорбцион', 'абхаз', 'ав', 'авангард', 'авантюрин', 'авар', 'аварийн', 'август', 'аверон', 'авиагоризонт', 'авиадвигател', 'авиакомпан', 'авиалин', 'авиапромышлен', 'авиастроен', 'авиац', 'авиацион', 'авиашин', 'авокад', 'авс', 'австр', 'австрал', 'авт', 'автобетононасос', 'автобетоносмесител', 'автоблокировк', 'автобус', 'автовышк', 'автогрейдер', 'автодорог', 'автодорожн', 'автоклав', 'автоклавн', 'автокомпонент', 'автокондиционер', 'автокра', 'автокресел', 'автокресл', 'автом', 'автомагнитол', 'автомалярн', 'автомастерск', 'автомат', 'автоматизац', 'автоматизирова', 'автоматик', 'автоматич', 'автоматическ', 'автоматн', 'автомашин', 'автомоб', 'автомобил', 'автомобилестроен', 'автомобильн', 'автономн', 'автопилот', 'ав

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [9]:
y_train = table['TNVED']
del table['TNVED']

In [16]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

x_training_data, x_test_data, y_training_data, y_test_data = train_test_split(X_TfidfVectorizer, y_train, test_size = 0.3)


In [30]:
model = KNeighborsClassifier(n_neighbors=30)


In [31]:
model.fit(X_TfidfVectorizer, y_train)

KNeighborsClassifier(n_neighbors=30)

In [29]:
# predictions = model.predict(x_test_data)
neibors2 = model.kneighbors(x_test_data[0:3000], return_distance=False)
# print(neibors2[2])
# print('res', y_test_data.iloc[2])
k = 0
for i, answer in enumerate(neibors2):
  # print(i, answer)
  for neibor in neibors2[i]:
    # print(neibor)
    # print(y_training_data.iloc[i], y_training_data.iloc[neibor])
    if y_test_data.iloc[i] == y_training_data.iloc[neibor]:
      k += 1
      break
result = float(k / len(neibors2))
print(result)

0.9306666666666666


In [32]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
predictions = model.predict(x_test_data[:2000])
print(classification_report(y_test_data[:2000], predictions))


              precision    recall  f1-score   support

         101       1.00      1.00      1.00         1
         106       1.00      1.00      1.00         1
         202       0.83      1.00      0.91         5
         203       0.67      0.67      0.67         3
         205       1.00      1.00      1.00         1
         206       1.00      1.00      1.00         3
         207       0.50      1.00      0.67         3
         301       0.00      0.00      0.00         1
         302       1.00      1.00      1.00         3
         303       0.67      1.00      0.80         4
         304       0.00      0.00      0.00         2
         305       1.00      1.00      1.00         4
         306       0.00      0.00      0.00         0
         307       1.00      0.50      0.67         4
         308       0.00      0.00      0.00         1
         401       1.00      1.00      1.00         1
         402       0.50      1.00      0.67         1
         403       1.00    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1

In [33]:
with open('indexes.json', 'w') as f:
  json.dump(y_train.to_list(), f)

In [ ]:
def demonstrate(num):
  print(table.iloc[num])
  neibors2 =  model.kneighbors([X_TfidfVectorizer.iloc[num]], return_distance=False)
  print(neibors2)
  for i, answer in enumerate(neibors2):
  # print(i, answer)
    stringa = ''
    for neibor in neibors2[i]:
      stringa = stringa + ' ' + str(y_training_data.iloc[neibor])
  
    print(stringa)

